## Template para uso de framework (scikit-learn)
En términos generales, debemos seguir los siguientes pasos:
1. Importar módulos
2. Cargar datos
3. Separar datos en subconjuntos
4. Entrenar el modelo
5. Analizar su desempeño
6. Usar el modelo para nuevas estimaciones (datos no vistos)


### 1. Importamos módulos

In [90]:
# Importar módulos
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_breast_cancer


### 2. Cargamos datos

In [91]:
# Cargar datos
data = load_breast_cancer()

#Convertimos array de datos a dataframe para visualizar mejor los datos
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = pd.Series(data.target)

X = df.drop(columns=['target'])
y = df['target']

### 3. Separar datos en subconjuntos

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=23)

### 4. Entrenar el modelo

In [93]:
model_1 = DecisionTreeClassifier(random_state=23,max_depth=100,criterion='gini',splitter='random')
model_1.fit(X_train,y_train)
y_pred1 = model_1.predict(X_test)
print(accuracy_score(y_test,y_pred1))

0.9651162790697675


In [94]:
model_2 = DecisionTreeClassifier(random_state=23,max_depth=100,criterion='entropy',splitter='random')
model_2.fit(X_train,y_train)
y_pred2 = model_2.predict(X_test)
print(accuracy_score(y_test,y_pred2))

0.9418604651162791


In [95]:
model_3 = DecisionTreeClassifier(random_state=23,max_depth=100,criterion='log_loss',splitter='random')
model_3.fit(X_train,y_train)
y_pred3 = model_3.predict(X_test)
print(accuracy_score(y_test,y_pred3))

0.9418604651162791


Vemos que a primera instancia, el modelo 1 es el que presenta mejor accuracy, cabe mencionar que se presentaron mejores resultados en todos los modelos cuando el splitter seleccionado era 'random' y no 'better', se hará el análisis de desempeño conforme al modelo 1.

### 5. Analizar desempeño

In [96]:
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95        27
           1       1.00      0.95      0.97        59

    accuracy                           0.97        86
   macro avg       0.95      0.97      0.96        86
weighted avg       0.97      0.97      0.97        86



In [97]:
print(confusion_matrix(y_test,y_pred1))

[[27  0]
 [ 3 56]]


El modelo tiene un buen rendimiento general con alta precisión y recall en ambas clases, el accuracy nos indica que el modelo pudo clasificar de forma correcta en el 97% de los casos lo cual es excelente, por otra parte se tuvieron unicamente 3 falsos negativos, lo cual puede ser peligroso debido a la naturaleza de los datos con los que se trabaja, sería mejor tener un modelo que a primera instancia clasifique falsos positivos que falsos negativos, sin embargo debido a que las clases a predecir están inbalanceadas con 2.5 más veces instancias de la clase 1, sería importante verificar que el alto rendimiento del modelo no se deba al desequilibrio de clases.

### 6. Usar el modelo para nuevas estimaciones

In [98]:
sub_dataset = pd.concat([X_test,y_test],axis=1)
sub_dataset = sub_dataset.sample(n=3,random_state=23)
sub_dataset.rename(columns={"target": "y_real"},inplace=True)

In [99]:
# Nuevas estimaciones
pred_puntuales = model_1.predict(sub_dataset.drop(columns=['y_real']))
sub_dataset['y_pred'] = pred_puntuales

In [100]:
sub_dataset

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,y_real,y_pred
411,11.04,16.83,70.92,373.2,0.10770,0.07804,0.03046,0.02480,0.1714,0.06340,...,79.93,471.4,0.1369,0.1482,0.1067,0.07431,0.2998,0.07881,1,1
409,12.27,17.92,78.41,466.1,0.08685,0.06526,0.03211,0.02653,0.1966,0.05597,...,89.00,610.2,0.1240,0.1795,0.1377,0.09532,0.3455,0.06896,1,1
274,17.93,24.48,115.20,998.9,0.08855,0.07027,0.05699,0.04744,0.1538,0.05510,...,135.10,1320.0,0.1315,0.1806,0.2080,0.11360,0.2504,0.07948,0,0


Podemos observar que para este caso las predicciones de las 3 instancias seleccionadas fueron correctas, esto da soporte a las conclusiones escritas en el reporte de resultados mostrados en el análisis de desempeño, donde se concluye que en el 93% de los casos las predicciones fueron correctas.